# **MUSIC THEORY CHATBOT**

#Connect Google Drive

In [1]:
# Google Drive mounted to access datasets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Importing necessary Setup and Dependencies

In [2]:
# Importing libraries
!pip install transformers torch gradio pandas scikit-learn nltk

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dropout
import kagglehub
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter
# Install dependencies as needed:
!pip install kagglehub[pandas-datasets] # Ensure this is run if not already


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

#Loading Dataset & Preprocess

In [3]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd

# Download the Roleplay dataset
_ = kagglehub.dataset_download("vampelium/roleplay-dataset")

# Load the JSON into a DataFrame
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "vampelium/roleplay-dataset",
    "roleplay_dataset.json"
)

# Show column names and sample data
print("Columns:", df.columns)
print("Sample row:", df.iloc[0])

/tmp/ipython-input-3-1718592312.py:9: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


Columns: Index(['prompt', 'response'], dtype='object')
Sample row: prompt                         You are a mischievous catgirl.
response    Silent as the night, deadly as the blade... Ju...
Name: 0, dtype: object


In [4]:
# Step 1: Load your DataFrame (you likely already have this from KaggleHub)
import pandas as pd

# If already loaded: df = pd.read_json(...) or from kagglehub
# Make sure it has 'prompt' and 'response' columns
print("Columns in df:", df.columns)

# Step 2: Combine prompt and response into a single chat line
df["chat_pair"] = df["prompt"] + " " + df["response"]

# Step 3: Save to train.txt in the /content directory (Colab default)
with open("/content/train.txt", "w", encoding="utf-8") as f:
    for line in df["chat_pair"]:
        f.write(line.strip() + "\n")

# Step 4: Confirm the file was written
!ls -lh /content/train.txt


Columns in df: Index(['prompt', 'response'], dtype='object')
-rw-r--r-- 1 root root 263M Jun 22 21:33 /content/train.txt


In [5]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Download the dataset and get the file path
path = kagglehub.dataset_download("vampelium/roleplay-dataset")

print("✅ Dataset downloaded to:", path)


✅ Dataset downloaded to: /kaggle/input/roleplay-dataset


In [6]:
import os

# List all files in the dataset directory
for root, dirs, files in os.walk(path):
    for file in files:
        print("Found file:", os.path.join(root, file))


Found file: /kaggle/input/roleplay-dataset/roleplay_dataset.json


In [7]:
import pandas as pd

# Load the data
df = pd.read_json("/kaggle/input/roleplay-dataset/roleplay_dataset.json")

# Format into alternating chatbot conversation format
with open("train.txt", "w", encoding="utf-8") as f:
    for idx, row in df.iterrows():
        prompt = row['prompt'].strip()
        response = row['response'].strip()
        f.write(f"User: {prompt}\nBot: {response}\n")


#Preprocessing


In [8]:
# Save conversational format for GPT-2
with open("train.txt", "w", encoding="utf-8") as f:
    for _, row in df.iterrows():
        prompt = row['prompt'].strip()
        response = row['response'].strip()
        f.write(f"User: {prompt}\nBot: {response}\n")


#Fine-Tune The Model

In [9]:
import pandas as pd
import os
import kagglehub

# Load the dataset
# Use the path obtained from kagglehub.dataset_download
# Assuming 'path' variable from previous cell holds the dataset root directory
json_file_path = os.path.join(path, "roleplay_dataset.json")

# Verify the file exists before trying to read it
if not os.path.exists(json_file_path):
    print(f"Error: JSON file not found at {json_file_path}")
    # Optionally, re-download the dataset here if needed
    path = kagglehub.dataset_download("vampelium/roleplay-dataset")
    json_file_path = os.path.join(path, "roleplay_dataset.json")
    if not os.path.exists(json_file_path):
         raise FileNotFoundError(f"JSON file still not found after re-download at {json_file_path}")


df = pd.read_json(json_file_path)

# Optional: limit rows for faster fine-tuning/testing
# df = df.sample(n=10000, random_state=42)

# Concatenate prompt and response into one text block
df["chat_pair"] = df["prompt"] + " " + df["response"]

# Save to train.txt
with open("train.txt", "w", encoding="utf-8") as f:
    for line in df["chat_pair"]:
        f.write(line.strip() + "\n")

In [10]:
with open("train.txt", "w", encoding="utf-8") as f:
    for line in df["chat_pair"]:
        f.write(line.strip() + "\n")


In [11]:
# Ensure necessary libraries are installed
!pip install datasets transformers torch

from datasets import load_dataset # Ensure this is from the 'datasets' library
from transformers import GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# Set the pad token to the eos token for GPT-2
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Method 1: Explicitly load using datasets.load_dataset
# This is the preferred method and *should* return a datasets.DatasetDict
try:
    # ✅ Load dataset with no caching to disk
    # Explicitly setting the type to 'text' and providing the data files
    dataset = load_dataset(
        "text",
        data_files={"train": "/content/train.txt"},
        keep_in_memory=True,
        cache_dir="/content/hf_cache",  # Needed to avoid default root
    )
    print(f"Successfully loaded dataset using datasets.load_dataset. Type: {type(dataset)}")

except Exception as e:
    print(f"Error loading dataset with datasets.load_dataset: {e}")
    print("Attempting to load manually using datasets.Dataset.from_text")
    # Method 2: If Method 1 fails or still results in TextDataset,
    # manually create a Dataset from the text file
    try:
        from datasets import Dataset
        with open("/content/train.txt", "r", encoding="utf-8") as f:
            lines = f.readlines()
        # Create a Dataset object from the list of lines
        # The 'text' column name is conventional for text datasets
        dataset = Dataset.from_dict({"text": lines})
        # Wrap it in a DatasetDict for consistency with Trainer
        dataset = {"train": dataset}
        print(f"Successfully created dataset manually. Type: {type(dataset)}")
    except Exception as e_manual:
        print(f"Error creating dataset manually: {e_manual}")
        raise # Re-raise the error if manual creation also fails


# Tokenize function
def tokenize_function(example):
    # Ensure that 'example' has the expected 'text' key from the loaded dataset
    # The 'text' key should be present if loaded using 'text' dataset type
    if "text" not in example:
        raise ValueError("Example dictionary does not contain 'text' key. Check dataset loading.")
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

# Check the type of the dataset *after* loading/creation
print(f"Final dataset type before tokenization: {type(dataset)}")

# Ensure dataset is a DatasetDict and access the 'train' split for mapping
if isinstance(dataset, dict) and "train" in dataset:
    print("Dataset is a DatasetDict with a 'train' key. Accessing the train split.")
    # Apply the map function to the 'train' split
    tokenized_datasets = dataset["train"].map(
        tokenize_function,
        batched=True,
        remove_columns=["text"]
    )
elif isinstance(dataset, Dataset):
     print("Dataset is a single Dataset object. Applying map directly.")
     # Apply the map function directly if it's a single Dataset
     tokenized_datasets = dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=["text"]
     )
else:
    # If after both loading attempts, the dataset is still not a DatasetDict or Dataset
    raise TypeError(f"Loaded dataset is not a DatasetDict or a Dataset object after loading/creation. Type: {type(dataset)}. Cannot apply map.")


# Collator for GPT2
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Training arguments
training_args = TrainingArguments(
    output_dir="./rp_model",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=500,
    logging_steps=50,
    save_total_limit=2,
    report_to=[],  # ⛔ Prevents logging to wandb or other integrations
)

# Train the model
# Pass the tokenized dataset object to the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets, # Pass the tokenized Dataset or DatasetDict split
    data_collator=data_collator,
)

print("\nStarting training...")
trainer.train()
print("Training finished.")

# Save the fine-tuned model and tokenizer
print("Saving model and tokenizer...")
trainer.save_model("./rp_model")
tokenizer.save_pretrained("./rp_model")
print("Model and tokenizer saved to ./rp_model")

# Show example from the *original* loaded dataset for context (if successful)
if isinstance(dataset, dict) and "train" in dataset and len(dataset["train"]) > 0:
    print("\nExample from original dataset:")
    print(dataset["train"][0])

# Show example from the *tokenized* dataset
if isinstance(tokenized_datasets, Dataset) and len(tokenized_datasets) > 0:
     print("\nExample from tokenized dataset:")
     print(tokenized_datasets[0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Map:   0%|          | 0/517861 [00:00<?, ? examples/s]


Starting training...


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,1.078800


KeyboardInterrupt: 

#Generating Chatbot Response

In [12]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import os # Import os module

# Load model
# Before loading, let's verify if the directory and expected files exist
model_dir = "./musicbot_model"

if not os.path.exists(model_dir):
    print(f"Error: Model directory '{model_dir}' not found.")
    # Consider re-running the training cell if the directory is missing
else:
    # Check for key tokenizer files
    vocab_file_exists = os.path.exists(os.path.join(model_dir, "vocab.json"))
    merges_file_exists = os.path.exists(os.path.join(model_dir, "merges.txt"))
    tokenizer_config_exists = os.path.exists(os.path.join(model_dir, "tokenizer_config.json"))

    if not (vocab_file_exists and merges_file_exists and tokenizer_config_exists):
        print(f"Warning: Not all required tokenizer files found in '{model_dir}'.")
        print(f"  vocab.json exists: {vocab_file_exists}")
        print(f"  merges.txt exists: {merges_file_exists}")
        print(f"  tokenizer_config.json exists: {tokenizer_config_exists}")
        print("Please ensure the previous cell executed successfully and saved the tokenizer files.")
    else:
        print(f"Required tokenizer files found in '{model_dir}'. Attempting to load.")
        try:
            tokenizer = GPT2Tokenizer.from_pretrained(model_dir)
            model = GPT2LMHeadModel.from_pretrained(model_dir)
            print("Model and tokenizer loaded successfully.")

            # Generate response function
            def chat_with_bot(user_input):
                prompt = f"User: {user_input}\nBot:"
                input_ids = tokenizer.encode(prompt, return_tensors="pt")
                # Ensure input_ids is on the same device as the model if using GPU
                if torch.cuda.is_available():
                    model.to('cuda')
                    input_ids = input_ids.to('cuda')

                with torch.no_grad():
                    output = model.generate(
                        input_ids,
                        max_length=100,
                        pad_token_id=tokenizer.eos_token_id,
                        do_sample=True,
                        temperature=0.7,
                        top_p=0.9
                    )
                response = tokenizer.decode(output[0], skip_special_tokens=True)
                return response.split("Bot:")[-1].strip()

            # Test
            print("\nTesting chatbot:")
            response = chat_with_bot("What is a major scale?")
            print(f"Bot: {response}")

        except Exception as e:
            print(f"An error occurred while loading the model or tokenizer: {e}")
            print("Please check the files in the directory and verify they are not corrupted.")

Error: Model directory './musicbot_model' not found.


#Chatbot User Interface

In [13]:
import gradio as gr

def respond(message):
    return chat_with_bot(message)

interface = gr.ChatInterface(fn=respond, title="🎵 MusicBot")
interface.launch()


/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1e5759eb793442ca5b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
